In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
movies_data = pd.read_csv('movies.csv')

Show Data

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.shape


(4803, 24)

In [5]:
# genere
#id
#keyword
#title
#overview
#cast 
#director
#index
new_movies_data = movies_data[['id','index','title','overview','genres','keywords','cast','director']]

In [6]:
new_movies_data.head()

,id,index,title,overview,genres,keywords,cast,director
0,19995,0,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,2,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,49026,3,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,4,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [7]:
new_movies_data.shape

(4803, 8)

In [8]:
new_movies_data.isnull().sum()

id            0
index         0
title         0
overview      3
genres       28
keywords    412
cast         43
director     30
dtype: int64

In [13]:
#new_movies_data.dropna(inplace=True)
for feature in new_movies_data:
    movies_data[feature] = movies_data[feature].fillna(' ')


In [14]:
new_movies_data.isnull().sum()

id          0
index       0
title       0
overview    0
genres      0
keywords    0
cast        0
director    0
dtype: int64

In [15]:
new_movies_data.shape

(4803, 8)

In [16]:
new_movies_data.duplicated().sum()

0

In [17]:
# combining all selected features
movies_feature = new_movies_data['overview']+' '+ new_movies_data['genres']+' '+ new_movies_data['keywords']+' '+ new_movies_data['cast']+' '+ new_movies_data['director']


# Using NLP

In [18]:
print(movies_feature)

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Length: 4803, dtype: object


In [19]:
# converting the text data into vector form usinng vectorization in NLP

vectorizer = TfidfVectorizer()

In [20]:
movies_vector =  vectorizer.fit_transform(movies_feature)


In [21]:
print(movies_vector)

  (0, 4207)	0.14134939369355703
  (0, 13932)	0.09005342709214369
  (0, 22773)	0.16033494305632573
  (0, 17493)	0.13142966756625357
  (0, 15307)	0.1833221037154887
  (0, 25464)	0.12399508683964489
  (0, 29137)	0.1611598924647531
  (0, 24418)	0.17001325611252707
  (0, 23212)	0.17974229527730723
  (0, 30041)	0.1611598924647531
  (0, 29650)	0.1951670832651365
  (0, 23253)	0.11818096973136863
  (0, 24837)	0.1438645563454754
  (0, 5492)	0.1853088451014517
  (0, 28995)	0.09351539472350909
  (0, 25016)	0.26454379242994663
  (0, 10596)	0.12441728045310148
  (0, 5183)	0.16672644228889758
  (0, 6477)	0.16033494305632573
  (0, 9842)	0.08530375463743382
  (0, 23607)	0.0850086095604104
  (0, 9576)	0.09113797346979599
  (0, 668)	0.07378222553110118
  (0, 530)	0.0639512332832129
  (0, 5149)	0.18147249173534674
  :	:
  (4802, 26724)	0.06809397308595068
  (4802, 18937)	0.0640212903261547
  (4802, 11719)	0.08180139518749079
  (4802, 29200)	0.07747795289570394
  (4802, 9175)	0.0768590349620255
  (4802, 19

Cosine Similarity

In [22]:
# finding similarity between movies using Cosine similarity Algorithm

similarity = cosine_similarity(movies_vector)

In [23]:
print(similarity)

[[1.         0.03405403 0.03366926 ... 0.0198114  0.0312866  0.0070467 ]
 [0.03405403 1.         0.03105138 ... 0.03489247 0.02038452 0.01381278]
 [0.03366926 0.03105138 1.         ... 0.02802907 0.05202542 0.01180564]
 ...
 [0.0198114  0.03489247 0.02802907 ... 1.         0.03743513 0.03653328]
 [0.0312866  0.02038452 0.05202542 ... 0.03743513 1.         0.03078044]
 [0.0070467  0.01381278 0.01180564 ... 0.03653328 0.03078044 1.        ]]


In [24]:
similarity.shape

(4803, 4803)

In [27]:
movie_name = input('Enter Your Fourite Movie Name : ')

Enter Your Fourite Movie Name : iron man


In [28]:
#creating the list of all movies in the data

list_of_all_movies = movies_data['title'].tolist()

In [29]:
print(list_of_all_movies)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [30]:
# finding the close match for movie enter by user

close_match = difflib.get_close_matches(movie_name,list_of_all_movies)

In [31]:
print(close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [32]:
find_close_match =  close_match[0]

In [33]:
print(find_close_match)

Iron Man


In [34]:
# finding the index of movie with title

index_of_movie =new_movies_data[new_movies_data.title == find_close_match]['index'].values[0]
print(index_of_movie)

68


In [35]:
# getting list of similar movies

similarity_score = list(enumerate(similarity[index_of_movie]))

In [36]:
print(similarity_score)

[(0, 0.056150579970167695), (1, 0.03401732994623017), (2, 0.012064859537367162), (3, 0.0071789326309206905), (4, 0.026603003762420873), (5, 0.015719424836411346), (6, 0.02085262428098871), (7, 0.18866378018547283), (8, 0.01184477139447955), (9, 0.03912055716867817), (10, 0.040525021459835955), (11, 0.014437581522556945), (12, 0.011486038774828247), (13, 0.011661904214599327), (14, 0.06273515298611317), (15, 0.03165258118360541), (16, 0.14469485758134), (17, 0.017406671392341977), (18, 0.018770686801887957), (19, 0.037049392317246126), (20, 0.039338557025078734), (21, 0.028044224275424315), (22, 0.00550954333617719), (23, 0.021185802057101723), (24, 0.014640483549900583), (25, 0.003967179496946251), (26, 0.1243153196610412), (27, 0.0331466050500315), (28, 0.0413690758845896), (29, 0.01041436560709311), (30, 0.08443891270886646), (31, 0.27983974060634287), (32, 0.02015903491851579), (33, 0.08276268282485758), (34, 0.0012739781566889024), (35, 0.022458594352442324), (36, 0.024073397075228

In [37]:
len(similarity_score)

4803

In [38]:
# sorting movies based on similarity

sorted_similar_movies = sorted(similarity_score , key = lambda x:x[1], reverse = True)

In [39]:
print(sorted_similar_movies)

[(68, 1.0), (79, 0.3302883516827091), (31, 0.27983974060634287), (7, 0.18866378018547283), (16, 0.14469485758134), (511, 0.1264411562923594), (26, 0.1243153196610412), (4401, 0.12274078684924541), (3623, 0.11469015253800395), (64, 0.10867604016639343), (182, 0.1008736209692106), (174, 0.09392055905517471), (783, 0.09335712384088662), (4033, 0.09320244217024788), (46, 0.09042783410277168), (94, 0.08842442376094754), (3466, 0.087260262478754), (420, 0.08465167684832094), (30, 0.08443891270886646), (203, 0.08345347304922132), (33, 0.08276268282485758), (3133, 0.0808497831319397), (1135, 0.08064965409153523), (2229, 0.0805813163572488), (2235, 0.08032223915006609), (138, 0.08031829062849051), (122, 0.07996630439577924), (788, 0.07929530356638335), (1233, 0.07889498763845794), (2487, 0.07763387710615863), (1282, 0.07741667254811552), (318, 0.07708053253259248), (725, 0.07704606684254396), (2186, 0.07695329588747561), (2625, 0.07682482953004208), (126, 0.07649730253196842), (2651, 0.07633820

In [40]:
# print the similar movies 

print("Movies Suggested For You : \n")


i = 1

for movies in sorted_similar_movies:
    index = movies[0]
    title_from_index =movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1

Movies Suggested For You : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . X-Men
7 . Captain America: Civil War
8 . The Helix... Loaded
9 . Made
10 . X-Men: Apocalypse
11 . Ant-Man
12 . The Incredible Hulk
13 . Mortdecai
14 . Super
15 . X-Men: Days of Future Past
16 . Guardians of the Galaxy
17 . Sliding Doors
18 . Hellboy II: The Golden Army
19 . Spider-Man 2
20 . X2
21 . X-Men: The Last Stand
22 . The Devil's Tomb
23 . Lord of War
24 . Machete Kills
25 . A Scanner Darkly
26 . The Last Airbender
27 . X-Men Origins: Wolverine
28 . Deadpool
29 . Get Rich or Die Tryin'


# Movie Recommender System

In [46]:
movie_name = input('Enter Your Fourite Movie Name : ')
list_of_all_movies = movies_data['title'].tolist()
close_match = difflib.get_close_matches(movie_name,list_of_all_movies)
find_close_match =  close_match[0]
index_of_movie =new_movies_data[new_movies_data.title == find_close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_movie]))
sorted_similar_movies = sorted(similarity_score , key = lambda x:x[1], reverse = True)

print("Movies Suggested For You : \n")


i = 1

for movies in sorted_similar_movies:
    index = movies[0]
    title_from_index =movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1

Enter Your Fourite Movie Name : avengers
Movies Suggested For You : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . Captain America: Civil War
6 . The Incredible Hulk
7 . X-Men
8 . Thor: The Dark World
9 . X-Men: Apocalypse
10 . Captain America: The First Avenger
11 . X-Men: Days of Future Past
12 . Ant-Man
13 . Iron Man 3
14 . Iron Man
15 . X-Men: The Last Stand
16 . Thor
17 . X2
18 . Man of Steel
19 . Kick-Ass
20 . The Amazing Spider-Man 2
21 . Superman II
22 . Guardians of the Galaxy
23 . The Shadow
24 . Serenity
25 . The Losers
26 . X-Men: First Class
27 . Fantastic 4: Rise of the Silver Surfer
28 . Deadpool
29 . The Spirit
